In [11]:
# Instalacion de dependecias del proyecto
!pip install ipykernel
!pippip install pydrive

!pip install tensorflow opencv-python scikit-learn numpy matplotlib 



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
"pippip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Define path data
path_data = "/gdrive/MyDrive/Bosco/garbage_dataset/garbage_classification/"

In [13]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Autenticación (esto abrirá tu navegador para iniciar sesión)
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

# Crear cliente de Google Drive
drive = GoogleDrive(gauth)

# Listar archivos de tu Drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in file_list:
    print(f'Título: {file["title"]} - ID: {file["id"]}')



ModuleNotFoundError: No module named 'pydrive'

In [ ]:
import os
import cv2
import numpy as np

def cargar_imagenes(dataset_path, map_categories, img_size=(224, 224)):

    categorias = map_categories.keys()

    imagenes = []
    etiquetas = []

    # Cargar imágenes por categoría
    for idx, categoria in enumerate(categorias):
      for garbage_type in map_categories[categoria]:
        categoria_path = os.path.join(dataset_path, categoria, garbage_type) # Contruyo la ruta para llegar al path donde se almacenan las imagenes
        for archivo in os.listdir(categoria_path): # Recorro las imagenes que hay en folder del garbage_type (Papel,plastico...)
            if archivo.endswith('.jpg') or archivo.endswith('.png'):
                img_path = os.path.join(categoria_path, archivo)
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)  # Redimensionar a 224x224
                img = img.astype('float32') / 255.0  # Normalizar a [0, 1]
                imagenes.append(img)
                etiquetas.append(idx)  # Etiquetar con el índice de la categoría

    return np.array(imagenes), np.array(etiquetas)

In [ ]:
import os

map_categories = {
    "APROVECHABLES" : os.listdir(f"{path_data}APROVECHABLES"),
    "NO_APROVECHABLE" : os.listdir(f"{path_data}NO_APROVECHABLE"),
    "ORGANICO" : os.listdir(f"{path_data}ORGANICO")

}

# Cargar las imágenes y etiquetas
imagenes, etiquetas = cargar_imagenes(path_data, map_categories)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(imagenes, etiquetas, test_size=0.2, stratify=etiquetas)

# Convertir las etiquetas a formato one-hot
y_train = to_categorical(y_train, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2

def crear_modelo():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Congelar las capas del modelo base
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Capa de pooling global
        Dropout(0.2),  # Para evitar sobreajuste
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # Tres categorías
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Crear y entrenar el modelo
modelo = crear_modelo()

# Resumen del modelo
modelo.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Usar early stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
modelo.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])


In [ ]:
# Evaluacion del modelo en bajo nivel
pérdida, precisión = modelo.evaluate(X_val, y_val)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_val_indices = np.argmax(y_val, axis=1)

# Hacemos predicciones
y_pred = modelo.predict(X_val)
y_pred_indices = np.argmax(y_pred, axis=1)

# Crear matriz de confusión
matriz_confusion = confusion_matrix(y_val_indices, y_pred_indices)

# Nombres de las clases
clases = map_categories.keys()

# Mostrar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusion, display_labels=clases)
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Matriz de Confusión")
plt.show()

In [ ]:
def predecir_imagen(imagen_path):
    img = cv2.imread(imagen_path) # <- Al integrar la camara me da directamente el objeto img, no necesito leer de un fichero
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Expande la dimensión para que sea compatible con el modelo

    prediccion = modelo.predict(img)
    categoria_idx = np.argmax(prediccion, axis=1)
    categorias = list(map_categories.keys())


    print(f"Predicción: {categorias[categoria_idx[0]]}")


In [ ]:
path_image_test = "/gdrive/MyDrive/Bosco/garbage_dataset/garbage_classification/cartyon.webp" #    <-Aqui pongo la ruta de la iamgen que deseo evaluar

# Ejemplo de predicción
predecir_imagen(path_image_test)

In [ ]:
modelo.save('/gdrive/MyDrive/Bosco/garbage_dataset/garbage_classification/modelo_mobilenetv2_entrenado.keras')